In [ ]:
import pandas as pd


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/busplainer-test-50/independent-security-50.xlsx')

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   commit-id      50 non-null     object
 1   commit-sha     50 non-null     object
 2   repo           33 non-null     object
 3   patch-content  30 non-null     object
dtypes: object(4)
memory usage: 1.7+ KB
None


In [ ]:
top_five = df.head(5)
top_five

,commit-id,commit-sha,repo,patch-content
0,d4d800c,d4d800ca1addc4141e03c5440a849bb64d1582cd,https://github.com/django/django,https://github.com/django/django/commit/d4d800...
1,c1edbc1,c1edbc1be78c1451b7549a94ba061bbc6e03a0f3,https://github.com/tensorflow/tensorflow,https://github.com/tensorflow/tensorflow/commi...
2,9f2deda,9f2deda929513ebde215422395692767c5d7fde6,https://github.com/python-pillow/Pillow,https://github.com/python-pillow/Pillow/commit...
3,ba908ae,ba908ae88d5925f4f6783eb234cc4ea95017472b,https://github.com/openstack/horizon,https://github.com/openstack/horizon/commit/ba...
4,2f10216,2f10216f84b55920de25422842a66260219e393f,https://github.com/django/django,https://github.com/django/django/commit/2f1021...


In [ ]:
import re
import requests

def extract_commit_message(row):
    patch_url = row['patch-content']
    if pd.isna(patch_url):
        return None

    try:
        response = requests.get(patch_url)
        if response.status_code == 200:
            patch_content = response.text
            pattern = r'^Subject: (?P<commit_message>.+?)(?=^---|\Z)'
            match = re.search(pattern, patch_content, re.DOTALL | re.MULTILINE)
            if match:
                return match.group('commit_message').strip()
    except Exception as e:
        print(f"Error fetching patch content from {patch_url}: {e}")
    return None

In [ ]:
# top_five.loc[:, 'commit-message'] = top_five['patch-content'].apply(extract_commit_message)
df['commit-message'] = df.apply(lambda row: extract_commit_message(row), axis=1)
# df['commit-message'] = df.apply(extract_commit_message, axis=1)

In [ ]:
df.to_excel('/content/drive/MyDrive/Colab Notebooks/busplainer-test-50/independent-security-50_updated_with_commit_message.xlsx', index=False)

Take 50 samples from the test set

In [ ]:
new_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/busplainer-test-50/test-sbt-random-explain.csv")

In [ ]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9270 entries, 0 to 9269
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   commit_sha      9270 non-null   object
 1   commit_node_id  9270 non-null   object
 2   commit_url      9270 non-null   object
 3   commit_message  9270 non-null   object
 4   sha             9270 non-null   object
 5   filename        9270 non-null   object
 6   contents_url    9270 non-null   object
 7   patch           9270 non-null   object
 8   author.date     8360 non-null   object
 9   committer.date  8360 non-null   object
 10  path            9270 non-null   object
 11  content         9270 non-null   object
 12  fix_commit_sha  9270 non-null   object
 13  sbt             9270 non-null   object
 14  repo            9270 non-null   object
dtypes: object(15)
memory usage: 1.1+ MB


In [ ]:
top_50_records = new_df[['commit_url', 'commit_message', 'content',]].head(50)

In [ ]:
top_50_records.to_csv('/content/drive/MyDrive/Colab Notebooks/busplainer-test-50/bugsplainer-top-50-records.csv', index=False)

In [ ]:
import json

with open('/content/drive/MyDrive/Colab Notebooks/pysecdb.json', 'r') as file:
    records = json.loads(file.read())

max_length = float('-inf')
min_length = 20000
count = 0
shorter_data = []

for record in records:
  if record["label"]=="security":
    content_length = len(str(record['content']))

    if content_length > max_length:
        max_length = content_length
    if content_length < min_length:
        # min_length = content_length
        count+=1
        shorter_data.append(record)

print("Maximum length:", max_length)
print("Minimum length:", min_length)
print(count)

with open('/content/drive/MyDrive/Colab Notebooks/pysecdb_shorter.json', 'w') as file:
    json.dump(shorter_data, file, indent=4)


Maximum length: 4379238
Minimum length: 20000
1174


In [ ]:
import random

random_shorter_data = random.sample(shorter_data, 100)

with open('/content/drive/MyDrive/Colab Notebooks/pysecdb_random_shorter.json', 'w') as file:
  json.dump(random_shorter_data, file, indent=4)

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/busplainer-test-50/test-sbt-random-explain.csv')

# json_data = df.to_json(orient='records')

# with open('output.json', 'w') as json_file:
#     json_file.write(json_data)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9270 entries, 0 to 9269
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   commit_sha      9270 non-null   object
 1   commit_node_id  9270 non-null   object
 2   commit_url      9270 non-null   object
 3   commit_message  9270 non-null   object
 4   sha             9270 non-null   object
 5   filename        9270 non-null   object
 6   contents_url    9270 non-null   object
 7   patch           9270 non-null   object
 8   author.date     8360 non-null   object
 9   committer.date  8360 non-null   object
 10  path            9270 non-null   object
 11  content         9270 non-null   object
 12  fix_commit_sha  9270 non-null   object
 13  sbt             9270 non-null   object
 14  repo            9270 non-null   object
dtypes: object(15)
memory usage: 1.1+ MB


In [ ]:
import random
import json

cropped_cols_data = df[['commit_sha', 'repo', 'commit_url']].sample(100)
print(cropped_cols_data[0:5])

                                    commit_sha               repo  \
92    d19973a357bdb942caf523cdfadefc4510bbe3dc      frappe.frappe   
8273  b6cc729a047c91e2efd469d3378a0688cdd42a28       dbiir.UER-py   
7887  5462216bb3d75472bedca2f8043116a4b53a2ff4  dbt-labs.dbt-core   
4823  60ccdf8a7a98c748833fa0044a22b5b71df80641   nonebot.nonebot2   
8666  b2d1d44668d44885d8a0ac496dd59136a8d1d1cc      espnet.espnet   

                                             commit_url  
92    https://api.github.com/repos/frappe/frappe/com...  
8273  https://api.github.com/repos/dbiir/UER-py/comm...  
7887  https://api.github.com/repos/dbt-labs/dbt-core...  
4823  https://api.github.com/repos/nonebot/nonebot2/...  
8666  https://api.github.com/repos/espnet/espnet/com...  


In [ ]:
json_list = json.loads(json.dumps(list(cropped_cols_data.T.to_dict().values())))
print(len(json_list))
print(json_list[0:5])

with open('/content/drive/MyDrive/Colab Notebooks/bugsplainer_100_random.json', 'w') as json_file:
    json.dump(json_list, json_file, indent=4)


100
[{'commit_sha': 'd19973a357bdb942caf523cdfadefc4510bbe3dc', 'repo': 'frappe.frappe', 'commit_url': 'https://api.github.com/repos/frappe/frappe/commits/d19973a357bdb942caf523cdfadefc4510bbe3dc'}, {'commit_sha': 'b6cc729a047c91e2efd469d3378a0688cdd42a28', 'repo': 'dbiir.UER-py', 'commit_url': 'https://api.github.com/repos/dbiir/UER-py/commits/b6cc729a047c91e2efd469d3378a0688cdd42a28'}, {'commit_sha': '5462216bb3d75472bedca2f8043116a4b53a2ff4', 'repo': 'dbt-labs.dbt-core', 'commit_url': 'https://api.github.com/repos/dbt-labs/dbt-core/commits/5462216bb3d75472bedca2f8043116a4b53a2ff4'}, {'commit_sha': '60ccdf8a7a98c748833fa0044a22b5b71df80641', 'repo': 'nonebot.nonebot2', 'commit_url': 'https://api.github.com/repos/nonebot/nonebot2/commits/60ccdf8a7a98c748833fa0044a22b5b71df80641'}, {'commit_sha': 'b2d1d44668d44885d8a0ac496dd59136a8d1d1cc', 'repo': 'espnet.espnet', 'commit_url': 'https://api.github.com/repos/espnet/espnet/commits/b2d1d44668d44885d8a0ac496dd59136a8d1d1cc'}]
